In [1]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import Pipeline

import joblib
import numpy as np
import pandas as pd
import time

In [2]:
df = pd.read_csv("./data/stratified_sample_data.csv")
df.head()

,password,strength
0,csillik,0.180594
1,huniihuu,0.177778
2,chaipy,0.172331
3,876876b,0.155556
4,miiwhy,0.154795
